In [3]:
import numpy as np
import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from tqdm import tqdm
import spacy

In [4]:
from spacy.matcher import Matcher
nlp = spacy.load("es_core_news_sm")
matcher = Matcher(nlp.vocab)

In [6]:
import os
import pandas as pd

# Ruta donde se encuentran los archivos CSV
ruta_archivos = "./"

# Listar todos los archivos que empiezan con "info279" y terminan en ".csv"
archivos_csv = [archivo for archivo in os.listdir(ruta_archivos) if archivo.startswith("info279") and archivo.endswith(".csv")]

# Lista para almacenar los DataFrames de cada archivo
dataframes = []

# Leer cada archivo y procesarlo
for archivo in archivos_csv:
    # Extraer el nombre del archivo sin la sigla "info279_"
    topico = archivo.replace("info279_", "").replace(".csv", "")
    
    # Leer el archivo CSV en un DataFrame
    df = pd.read_csv(os.path.join(ruta_archivos, archivo))
    
    # Seleccionar solo las columnas "title" y "text"
    df_seleccionado = df[["title", "text"]]
    
    # Añadir una columna "topico" con el nombre del archivo procesado
    df_seleccionado["topico"] = topico
    
    # Añadir el DataFrame a la lista
    dataframes.append(df_seleccionado)

# Concatenar todos los DataFrames en uno solo
df_final = pd.concat(dataframes, ignore_index=True)

# Guardar el DataFrame final en un archivo CSV
df_final.to_csv("noticias_etiquetadas.csv", index=False)

print("Archivo 'noticias_etiquetadas.csv' creado con éxito.")


C:\Users\Tuki\AppData\Local\Temp\ipykernel_23140\2011106929.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seleccionado["topico"] = topico
C:\Users\Tuki\AppData\Local\Temp\ipykernel_23140\2011106929.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seleccionado["topico"] = topico
C:\Users\Tuki\AppData\Local\Temp\ipykernel_23140\2011106929.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



Archivo 'noticias_etiquetadas.csv' creado con éxito.


In [7]:
dataset = pd.read_csv("noticias_etiquetadas.csv")
dataset

,title,text,topico
0,"Presentan la Estrategia Regional de Ciencia, T...",En el Distrito de Innovación V21 en Viña del ...,cienciatecnologia
1,"Ciencia, tecnología, conocimiento e innovación...","miércoles, 14 junio, 2023 a las 21:29 Editor A...",cienciatecnologia
2,Región de Coquimbo contará con el primer parqu...,El Polo Científico y Tecnológico de la Univers...,cienciatecnologia
3,Armada invita a investigadores a realizar cien...,"Fortalecer el sistema de Ciencia, Tecnología, ...",cienciatecnologia
4,Silvia Díaz es designada para dirigir el Conse...,"El Presidente de la República, Gabriel Boric, ...",cienciatecnologia
...,...,...,...
108697,“Vine aquí a desaparecer del mundo”: la peligr...,/ NoticiasInformación basada en hechos y verif...,sociedad
108698,"Revilla, a la sombra del tiempo: ""Me da más mi...",EL ESPAÑOL visita al presidente cántabro en su...,sociedad
108699,Streaming y entrevistas: La programación de es...,"#nota_tabla_emol .tablaennoticia tr td,#nota_...",sociedad
108700,Formulario de búsqueda,Estamos donde tú estás. Síguenos en: */.not-l...,sociedad


In [11]:

# Obtén 1000 filas aleatorias del DataFrame
random_rows = dataset.sample(n=1000, random_state=42)
# Encuentra las clases únicas en la columna "topico"
topicounico = random_rows["topico"].unique()


In [12]:
topicounico

array(['entretenimiento', 'medioambiente', 'cienciatecnologia',
       'politica', 'deportes', 'sociedad', 'cultura', 'salud', 'economia'],
      dtype=object)

In [13]:
# Crea la lista de tuplas con las etiquetas de clases
data_list = []
for _, row in random_rows.iterrows():
    title = row["title"]
    topico = row["topico"]
    cats = {cls: 1 if cls == topico else 0 for cls in topicounico}
    data_list.append((title, {"cats": cats}))

In [14]:
data_list

[('Taylor Swift logra con su película de “Eras Tour” romper el récord de preventa en los cines',
  {'cats': {'entretenimiento': 1,
    'medioambiente': 0,
    'cienciatecnologia': 0,
    'politica': 0,
    'deportes': 0,
    'sociedad': 0,
    'cultura': 0,
    'salud': 0,
    'economia': 0}}),
 ('Los pastores de Trump llegan a la Brasilia de Bolsonaro',
  {'cats': {'entretenimiento': 0,
    'medioambiente': 1,
    'cienciatecnologia': 0,
    'politica': 0,
    'deportes': 0,
    'sociedad': 0,
    'cultura': 0,
    'salud': 0,
    'economia': 0}}),
 ('[Opinión] Silvio Becerra: El trabajo de ayer y hoy',
  {'cats': {'entretenimiento': 0,
    'medioambiente': 0,
    'cienciatecnologia': 1,
    'politica': 0,
    'deportes': 0,
    'sociedad': 0,
    'cultura': 0,
    'salud': 0,
    'economia': 0}}),
 ('Ministra Aguilera por crisis de Isapres y gestión del Ejecutivo: "La Ley Corta da viabilidad a las sentencias"',
  {'cats': {'entretenimiento': 0,
    'medioambiente': 0,
    'cienciatec

In [15]:
import spacy
import random
from spacy.training.example import Example
from spacy.util import minibatch

from spacy.pipeline.tok2vec import DEFAULT_TOK2VEC_MODEL

In [16]:
# Datos de entrenamiento
TRAIN_DATA = data_list
nlp = spacy.load("es_core_news_sm")

In [17]:
train_examples = []

for example in TRAIN_DATA:
    train_examples.append(Example.from_dict(nlp.make_doc(example[0]), example[1]))

def get_examples():
    return train_examples

In [18]:
model = {
            "@architectures": "spacy.TextCatCNN.v2",
            "exclusive_classes": True,
            "tok2vec": DEFAULT_TOK2VEC_MODEL,
        }

In [19]:
# Crear un componente TextCategorizer con un modelo CNN
textcat = nlp.add_pipe("textcat", config={"model": model})
textcat.initialize(get_examples)

In [20]:
from spacy.util import compounding

# Entrenar el modelo
with nlp.select_pipes(enable="textcat"):
    optimizer = nlp.begin_training()
    for epoch in range(100):
        losses = {}
        random.shuffle(TRAIN_DATA)
        # Dividir los datos en lotes y actualizar el modelo
        for batch in minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001)):
            texts, annotations = zip(*batch)
            example = []
            # Actualizar el modelo con iteraciones
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
            nlp.update(example, drop=0.5, losses=losses)
        print(losses)

{'textcat': 23.727783679962158}
{'textcat': 23.27928974479437}
{'textcat': 21.820234432816505}
{'textcat': 20.176081229001284}
{'textcat': 17.714884192682803}
{'textcat': 15.708562945597805}
{'textcat': 13.78350126058649}
{'textcat': 11.693946906903875}
{'textcat': 10.042453624191694}
{'textcat': 8.1272634383688}
{'textcat': 7.079313470607815}
{'textcat': 6.08074080387442}
{'textcat': 5.01868396906942}
{'textcat': 4.147228656347325}
{'textcat': 4.4746784724572635}
{'textcat': 3.920192777763713}
{'textcat': 2.950111745355061}
{'textcat': 2.710390826378878}
{'textcat': 2.591068446895613}
{'textcat': 1.7896809600360744}
{'textcat': 2.163098542733532}
{'textcat': 2.08930721853563}
{'textcat': 2.054041830242989}
{'textcat': 1.831676732245638}
{'textcat': 1.395453095042845}
{'textcat': 1.5148856500011298}
{'textcat': 1.696810678709106}
{'textcat': 1.3099524571367804}
{'textcat': 1.4989307917688661}
{'textcat': 1.4227069052601493}
{'textcat': 1.6548165914336666}
{'textcat': 1.3457298253873917

In [21]:
# Guardar el modelo entrenado en disco
nlp.to_disk("model_1000_CNNv2")

# Cargar el modelo entrenado
nlp_loaded = spacy.load("model_1000_CNNv2")#modelo_clasificador_noticias")

In [22]:
# Realizar predicciones en nuevos titulares
nuevos_titulares = [
    "El partido de fútbol se jugará mañana",
    "Nueva legislación de impuestos aprobada por el gobierno",
    "Nueva aplicación móvil para seguimiento de ejercicios",
    "Grave accidente de tráfico en la autopista deja varios heridos",
    "Avión se estrella en zona montañosa, sobreviven todos los pasajeros",
    "Incendio en fábrica causa evacuación masiva en la ciudad",
    "Naufragio de embarcación turística en el mar, varios desaparecidos",
    "Accidente de tren en la estación central, servicios suspendidos",
    "Nuevas regulaciones para reducir las emisiones de gases contaminantes en la industria",
    "Los científicos advierten sobre el aumento del nivel del mar debido al deshielo de los polos",
    "Proyecto de reforestación masiva busca combatir la deforestación en la Amazonia",
    "La contaminación del aire en las ciudades alcanza niveles peligrosos para la salud",
    "Descubren una nueva especie en peligro de extinción en la selva tropical",
    "Acuerdo internacional para proteger los océanos y conservar la vida marina",
    "Innovadora tecnología de reciclaje convierte plásticos en materiales reutilizables",
    "Cambio climático: las temperaturas extremas se vuelven más frecuentes y devastadoras",
    "Los esfuerzos de conservación salvan a una especie en peligro crítico de desaparecer",
    "La energía solar y eólica supera a los combustibles fósiles como fuente de electricidad",


]

for titular in nuevos_titulares:
    doc = nlp_loaded(titular)
    print(f"Titular: {titular}")
    print("Categorías predichas:")
    for label, score in doc.cats.items():
        print(f"{label}: {score:.4f}")

Titular: El partido de fútbol se jugará mañana
Categorías predichas:
entretenimiento: 0.0000
medioambiente: 0.0000
cienciatecnologia: 0.0000
politica: 0.0000
deportes: 1.0000
sociedad: 0.0000
cultura: 0.0000
salud: 0.0000
economia: 0.0000
Titular: Nueva legislación de impuestos aprobada por el gobierno
Categorías predichas:
entretenimiento: 0.0001
medioambiente: 0.0000
cienciatecnologia: 0.0004
politica: 0.0013
deportes: 0.0000
sociedad: 0.0013
cultura: 0.0000
salud: 0.0303
economia: 0.9667
Titular: Nueva aplicación móvil para seguimiento de ejercicios
Categorías predichas:
entretenimiento: 0.0077
medioambiente: 0.0725
cienciatecnologia: 0.0286
politica: 0.0000
deportes: 0.0031
sociedad: 0.2482
cultura: 0.0008
salud: 0.5804
economia: 0.0587
Titular: Grave accidente de tráfico en la autopista deja varios heridos
Categorías predichas:
entretenimiento: 0.0000
medioambiente: 0.0000
cienciatecnologia: 0.0000
politica: 0.0000
deportes: 0.9994
sociedad: 0.0000
cultura: 0.0000
salud: 0.0006
ec